# Imports

In [1]:
from datasets import load_dataset
from collections import Counter
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel

# linea que arregla algunos errores de loadeo de datasets
#pip install --upgrade datasets

# linea que corre torch acelerado en mac
# pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

/Users/malepersonal/Git/Puntualizacion-Capitalizacion/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/malepersonal/Git/Puntualizacion-Capitalizacion/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [2]:
linux = False
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: mps


# Ejericio b)

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [3]:
questions, question_for_mixture = get_questions()

Se descargaron 5000 preguntas en Español.


### Fuente 2: Dataset provisto para Notebook 10

In [4]:
oraciones_rnn = get_notebook_dataset()

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


### Fuente 3: Dataset sintetico generado con Gemini

In [5]:
oraciones_sinteticas = get_gemini_dataset()

Hay 1413 oraciones sintéticas.


### Fuente 4: Articulos de Wikipedia

In [6]:
frases_wikipedia = get_wikipedia_dataset()

['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']


### Fuente 5: Subtitulos de peliculas

In [7]:
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['Claro.', 'Que no. Ojalá se te atragante la comida y te tengan que llevar al hospital medio ahogado, mirá.', '¿Cómo? Tengo que poner la edad justa.', 'Preguntó por Benigno, ese amigo tuyo que es tan importante.', 'Sí, soy la esposa. ¿Qué? ¡¿Qué?! ¡¿Pero qué dice?! Pero, señor, usted dice cada cosa que... Además hable claro. ¡Hable claro! Bueno sáquese lo que tiene en la boca y hable claro. Repítamelo, quiere. ¿Pero es un chiste? Ah, no sería la primera... No sería la primera vez. No le da vergüenza en un Pero señor: no oigo. No lo que pasa es que yo no me fijé. Pero si los propios hijos, que son la sangre de su sangre, la carne de su carne, no se dieron cuenta...', 'Seguramente mamá Cora debe estar en lo de Emilia y los hombres deben haber ido a la rotisería a comprar comida.', 'Y bueno, te querrá ayudar.', 'Matildlta: traé las masltas, querida, para c

### Fuente 6: Mixture of preguntas y afirmaciones

In [8]:
import unicodedata


cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)

['¿De qué ha sido acusado Bill Cosby? La resolución de la ONU fue vetada por un miembro permanente.',
 'Carmen, ¿sabes si va a llover esta tarde en la ciudad? ¿Cómo se abrevia comúnmente el nombre de la catedral de San Esteban?',
 'La ciudad de Cartago en Túnez fue una gran potencia marítima. ¿Dónde se darán cita los representantes de ambos grupos?',
 '¿Cuándo fue derrotada Amida? El anuncio de Coca-Cola era muy emotivo.',
 '¿Qué qué color es la leyenda de Hollywood? ¿Cómo es el clima de Villahermosa?']

## Juntamos las fuentes

In [9]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones de mixture:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones de mixture: 4239
Algunas oraciones aleatorias:


['¿Qué ciudades inglesas fracasaron en su tentativa de albergar unos Juegos Olímpicos? La cumbre de la OTAN se centró en la amenaza de Rusia.',
 'Valencia es el mejor 10 y debe jugar, sin desmerecer a Alonso ni a Villa.',
 'El 25 de febrero de 1912, Jung fundó la Sociedad de Intereses Psicoanalíticos y con este acto formalizó el camino hacia su propia versión del psicoanálisis.',
 'Pulp Fiction en Rotten Tomatoes (en inglés).',
 'Conducir un BMW es una gran experiencia.']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

In [4]:
from train.RNN import PunctuationCapitalizationRNN

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

# Congelar todo primero
for param in bert_model.parameters():
    param.requires_grad = False

# Congelar embeddings explícitamente (ya congelados, pero para mayor claridad)
for param in bert_model.embeddings.word_embeddings.parameters():
    param.requires_grad = False

# Descongelar últimas N capas
N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

# Descongelar pooler
for param in bert_model.pooler.parameters():
    param.requires_grad = True


# Crear el modelo
model = PunctuationCapitalizationRNN(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_start_classes=len(PUNCT_START_TAGS),
    num_punct_end_classes=len(PUNCT_END_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)


# le quiero pasar el weight param al criterion para mejorar el desbalanceo de clases en base a un counter de tokens
# Contar ocurrencias de cada etiqueta de puntuación y capitalización
punct_start_counter = Counter()
punct_end_counter = Counter()
cap_counter = Counter()
for input_ids, attention_mask, punct_start_labels, punct_end_labels, cap_labels in dataloader_train:
    punct_start_np = punct_start_labels.cpu().numpy()
    punct_end_np   = punct_end_labels.cpu().numpy()
    cap_np         = cap_labels.cpu().numpy()

    # Contar etiquetas válidas (ignorando -100)
    valid_start = punct_start_np[punct_start_np != -100]
    valid_end   = punct_end_np[punct_end_np != -100]
    valid_cap   = cap_np[cap_np != -100]

    punct_start_counter.update(valid_start)
    punct_end_counter.update(valid_end)
    cap_counter.update(valid_cap)

# como las clases estan desbalanceadas, calculamos los pesos inversos
total_start = sum(punct_start_counter.values())
total_end   = sum(punct_end_counter.values())
total_cap   = sum(cap_counter.values())
beta = 0.7

start_weights = {
    tag: (total_start / count)**beta
    for tag, count in punct_start_counter.items()
}
end_weights = {
    tag: (total_end / count)**beta
    for tag, count in punct_end_counter.items()
}
cap_weights = {
    tag: (total_cap / count)**beta
    for tag, count in cap_counter.items()
}

# Tensores
start_weights_tensor = torch.tensor(
    [start_weights.get(i, 1.0) for i in range(len(PUNCT_START_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)

end_weights_tensor = torch.tensor(
    [end_weights.get(i, 1.0) for i in range(len(PUNCT_END_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)

cap_weights_tensor = torch.tensor(
    [cap_weights.get(i, 1.0) for i in range(len(CAP_TAGS))],
    dtype=torch.float32
).to(device).clamp(min=1.0, max=5.0)


criterion_start = nn.CrossEntropyLoss(ignore_index=-100, weight=start_weights_tensor)
criterion_end   = nn.CrossEntropyLoss(ignore_index=-100, weight=end_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)

trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.rnn.parameters()) \
  + list(model.punct_start_classifier.parameters()) \
  + list(model.punct_end_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)
# Entrenamiento 
train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion_start=criterion_start,criterion_end=criterion_end, criterion_cap = criterion_cap, device=device, epochs=3)

NameError: name 'dataloader_train' is not defined

ssssss

## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

In [ ]:
entrada = "hola como estas"
print(f"{entrada} => {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

## Prueba de control de Overfitting

explicacion de del corro

In [3]:
frases = ["¿QUE?"]
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_loader = get_dataloader(frases, max_length=25, batch_size=1, device=device,tokenizer=tokenizer)

model = PunctuationCapitalizationRNN(
    bert_model=bert_model,
    hidden_dim=64,
    num_punct_start_classes=5,
    num_punct_end_classes=5,
    num_cap_classes=4
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)  # Alto LR

criterion_punct_start = nn.CrossEntropyLoss(ignore_index=-100)
criterion_punct_end   = nn.CrossEntropyLoss(ignore_index=-100)
criterion_cap         = nn.CrossEntropyLoss(ignore_index=-100)

train(
    model,
    dataloader_train=train_loader,
    dataloader_test=train_loader,
    optimizer=optimizer,
    criterion_start=criterion_punct_start,
    criterion_end=criterion_punct_end,
    criterion_cap=criterion_cap,
    device=device,
    epochs=200
)

entrada = "que"
print("Predicción:", predict_and_reconstruct(model, entrada, tokenizer, device))


NameError: name 'PunctuationCapitalizationRNN' is not defined

## Export modelo

In [ ]:
torch.save(model, "modelo_fine_tuned.pt")

In [ ]:
torch.save(model.state_dict(), "modelo_fine_tuned_state_dict.pt")

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## Evaluacion modelo entrenado

In [5]:
# Cargar la clase del modelo
from train.PunctuationCapitalizationRNNBidirectional import PunctuationCapitalizationRNNBidirectional

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

# Crear la instancia
model = PunctuationCapitalizationRNNBidirectional(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

# Cargar los pesos en CPU
state_dict = torch.load("models/modelo_fine_tuned_state_dict_bidirec.pt", map_location=device)
model.load_state_dict(state_dict)
model.eval()

ModuleNotFoundError: No module named 'train.PunctuationCapitalizationRNNBidirectional'

In [7]:
entrada = "hola como estas"
print(f"{entrada} => {predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

hola como estas => Hola, como estas.


## Generacion CSV TP

In [23]:
from utils.datautils import *

In [5]:
# Ruta a un archivo TXT con párrafos (una instancia por párrafo)
ruta_txt = "predict/prueba.txt"

# Ejecutar predicciones y guardar CSV
df_predicciones = predicciones_TP(ruta_txt, model, tokenizer, device, max_length=128, verbose=True)

# Mostrar las primeras filas del dataframe con las predicciones
print(df_predicciones)


Instancia 1: has visto la última película de marvel es increíble
Token ID | Token           | Punt Inicial | Punt Final | Capitalizacion
----------------------------------------------------------------------
       1 | has             | Ø            | ,          | init         
       2 | visto           | Ø            | ,          | upper        
       3 | la              | Ø            | ,          | upper        
       4 | última          | Ø            | ,          | upper        
       5 | película        | Ø            | ,          | upper        
       6 | de              | Ø            | ,          | upper        
       7 | mar             | Ø            | ,          | upper        
       8 | ##vel           | Ø            | ,          | upper        
       9 | es              | Ø            | ,          | upper        
      10 | in              | Ø            | ,          | upper        
      11 | ##cre           | Ø            | ,          | upper        
      12 |